In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm

rng = np.random.RandomState(123)


# Learn To Shoot

Target on the wall above april tags.

projectile is an approximatly 12in diameter donut.

Robot on feild, with angeled arm, at end of arm are in-line flaps

Robot has limelight vision above the front bumper. it can identify april tags and report the ???

In [1]:
with pm.Model() as generate:
    # Priors


    #Data mutable
    Ad = pm.MutableData(rng.uniform(0, 100, 100))
    Ah = pm.MutableData(rng.uniform(0, 100, 100))  # height of april tag in meters
    Vo = pm.Uniform('Vo', lower=0, upper=100)
    #theta = pm.Uniform('theta', lower=-np.pi(), upper=np.pi(), transform='circular')
    alphaA = pm.Uniform('alphaA', lower=-np.pi(), upper=np.pi(), transform='circular')
    betaF = pm.Normal('BetaF', mu=0, sd=1) #BetaF is the conv to effective flap angle of the flaps from the center line of the shooter; assuming equal on both sides
    alphaF = pm.Uniform('alphaF', lower=-np.pi(), upper=np.pi(), transform='circular')
    alphaR = pm.Uniform('alphaR', lower=-np.pi(), upper=np.pi(), transform='circular')

    # use pathagorean theorem to solve for ground distance from april tag
    # d = sqrt(Ad^2 - Ah^2); Ad is straight line hypthosnuse, Ah is height of april tag
    d = pm.math.sqrt(Ad**2 - Ah**2)
    # trajectory equation solved for time
    # d = Vo * cos(theta) * t
    t = d / (Vo * pm.math.cos(alphaA))
    # trajectory equation solved for height
    # h = Vo * sin(theta) * t - 0.5 * g * t**2
    h = Vo * pm.math.sin(alphaA) * t - 0.5 * g * t**2

    #Shooter height using lenght of arm (La) and angle of arm (alphaA)
    Sh = La * pm.math.sin(alphaA)
    Th_sim = h + Sh

    #angular error to lead to horizontal error
    #Talpha is the angle of the target from the center of the shooter
    alphaFprime = betaF * alphaF * (1- alphaA/np.pi())
    Talpha = alphaR + alphaFprime




NameError: name 'pm' is not defined